In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2


In [2]:
DATA_DIR = "/Users/krishrana/Python/Dataset" #path to the dataset your 

CATEGORIES = ["Car", "Auto", "Motorcycle"] #categories of vehicles

img_size=70

training_data = []
def create_training_data():
	#iterating through different categories
	for category in CATEGORIES:
		path = os.path.join(DATA_DIR,category) #creating the path to the images
		print(path)
		class_num = CATEGORIES.index(category) #creating classification label for the dataset (indexing the categories)
		print(class_num) 
		for img in os.listdir(path): #loading each image in the directory
			try:
				img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)
				new_array = cv2.resize(img_array,(img_size,img_size)) #resizeing the all the images to same size(70,70)
				training_data.append([new_array,class_num])
			except Exception as e:
				pass


In [3]:
create_training_data()
print(len(training_data))

/Users/krishrana/Python/Dataset/Car
0
/Users/krishrana/Python/Dataset/Auto
1
/Users/krishrana/Python/Dataset/Motorcycle
2
1326


In [4]:
import random
random.shuffle(training_data) #randomizing the data

In [5]:
X=[]
y=[]

for features,label in training_data: #new_array is features and class_num is label
	X.append(features)
	y.append(label)


X=np.array(X).reshape(-1,img_size,img_size,1) #transforming X into numpy array
print(X.shape)
print(len(y))

(1326, 70, 70, 1)
1326


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras import optimizers

Using TensorFlow backend.


In [7]:
X=X/255.0
model=Sequential()
model.add(Conv2D(32,(3,3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))


model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))



model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))




model.add(Flatten())  #makes it 1d array
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(3))
model.add(Activation('softmax'))

model.summary()

W0107 19:29:30.952189 4670182848 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 68, 68, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 68, 68, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 66, 66, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 66, 66, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 33, 33, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 31, 31, 64)        0

In [8]:
adam=optimizers.Adam(lr=0.01)
batch_size=16
epochs=15

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
target = to_categorical(y, num_classes=3)
print(target[:5, :])

[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [10]:
model.fit(X, target, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Train on 1060 samples, validate on 266 samples
Epoch 1/15
1060/1060 [==============================] - 9s 8ms/sample - loss: 0.9015 - acc: 0.5170 - val_loss: 0.7559 - val_acc: 0.7143
Epoch 2/15
1060/1060 [==============================] - 7s 7ms/sample - loss: 0.6366 - acc: 0.6849 - val_loss: 0.5704 - val_acc: 0.7180
Epoch 3/15
1060/1060 [==============================] - 7s 7ms/sample - loss: 0.5873 - acc: 0.7151 - val_loss: 0.5698 - val_acc: 0.6917
Epoch 4/15
1060/1060 [==============================] - 7s 7ms/sample - loss: 0.4755 - acc: 0.7821 - val_loss: 0.4920 - val_acc: 0.7744
Epoch 5/15
1060/1060 [==============================] - 7s 7ms/sample - loss: 0.3247 - acc: 0.8774 - val_loss: 0.2982 - val_acc: 0.8797
Epoch 6/15
1060/1060 [==============================] - 8s 7ms/sample - loss: 0.2365 - acc: 0.9142 - val_loss: 0.2727 - val_acc: 0.8947
Epoch 7/15
1060/1060 [==============================] - 8s 7ms/sample - loss: 0.1734 - acc: 0.9349 - val_loss: 0.3806 - val_acc: 0.8571
E

In [11]:
model.save('vehicleNet.pt')

In [12]:
def prepare(filepath):
    IMG_SIZE = 70  # 
    test_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    test_array = cv2.resize(test_array, (IMG_SIZE, IMG_SIZE))
    return test_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


In [13]:
test_path=input()
test_image=prepare(test_path)
pre=model.predict(test_image)
y = np.argmax(pre, axis=-1)
if (y==0):
    print('Car')
elif(y==1):
    print('Auto')
else:
    print('Motorcycle')
#print(test_image.shape)

/Users/krishrana/Desktop/jkasv.png
Car
